## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 🛑 Middleware Termination

## Industry Use Case: Transaction Compliance Screening

This notebook demonstrates how middleware can **terminate** the execution pipeline early.

| Feature | FSI Application |
|---------|-----------------|
| **Pre-Termination** | Block prohibited transaction types |
| **Post-Termination** | Rate limit transaction requests |
| **Short-Circuit** | Return immediate compliance responses |

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

print(f"✅ Environment loaded")

In [ ]:
from collections.abc import Awaitable, Callable

from agent_framework import (
    AgentMiddleware,
    AgentRunContext,
    AgentResponse,
    ChatMessage,
    Role,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

print("✅ All imports loaded")

## Define Transaction Tool Function

In [ ]:
def check_transaction(transaction_id: str) -> str:
    """Check the status of a transaction."""
    return f"Transaction {transaction_id} is approved and completed."

print("✅ Tool defined: check_transaction")

## Termination Middleware Classes

**PreTerminationMiddleware**: Blocks requests with prohibited transaction types BEFORE processing

**PostTerminationMiddleware**: Allows processing but limits total requests (rate limiting)

In [ ]:
class PreTerminationMiddleware(AgentMiddleware):
    """Terminates execution BEFORE agent processing for prohibited terms."""

    def __init__(self, blocked_words: list[str]):
        self.blocked_words = [word.lower() for word in blocked_words]

    async def process(
        self,
        context: AgentRunContext,
        next: Callable[[AgentRunContext], Awaitable[None]],
    ) -> None:
        last_message = context.messages[-1] if context.messages else None
        if last_message and last_message.text:
            query = last_message.text.lower()
            for blocked_word in self.blocked_words:
                if blocked_word in query:
                    print(f"[PreTermination] ⛔ Blocked: '{blocked_word}' detected")
                    context.result = AgentResponse(
                        messages=[
                            ChatMessage(
                                role=Role.ASSISTANT,
                                text=f"Cannot process requests containing '{blocked_word}'. Please contact compliance.",
                            )
                        ]
                    )
                    context.terminate = True
                    return  # Don't call next()

        await next(context)


class PostTerminationMiddleware(AgentMiddleware):
    """Allows processing but terminates after max responses (rate limiting)."""

    def __init__(self, max_responses: int = 2):
        self.max_responses = max_responses
        self.response_count = 0

    async def process(
        self,
        context: AgentRunContext,
        next: Callable[[AgentRunContext], Awaitable[None]],
    ) -> None:
        if self.response_count >= self.max_responses:
            print(f"[PostTermination] ⛔ Rate limit reached ({self.max_responses} requests)")
            context.terminate = True
            return

        await next(context)
        self.response_count += 1
        print(f"[PostTermination] Request {self.response_count}/{self.max_responses}")


print("✅ Middleware classes defined")

## Example 1: Pre-termination (Compliance Blocking)

Blocks requests containing prohibited transaction types.

In [ ]:
async def run_pre_termination_example():
    """Pre-termination blocks prohibited requests."""
    print("\n--- Example 1: Pre-termination Middleware ---\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="TransactionAgent",
            instructions="You help with transaction inquiries.",
            tools=[check_transaction],
            middleware=[PreTerminationMiddleware(blocked_words=["offshore", "anonymous"])],
        ) as agent,
    ):
        # Normal query
        print("1. Normal query:")
        result = await agent.run("Check transaction TXN-12345")
        print(f"   Agent: {result.text}\n")

        # Blocked query
        print("2. Blocked query (contains 'offshore'):")
        result = await agent.run("How do I set up offshore transfers?")
        print(f"   Agent: {result.text}\n")

await run_pre_termination_example()

## Example 2: Post-termination (Rate Limiting)

Allows initial requests but terminates after max responses.

In [ ]:
async def run_post_termination_example():
    """Post-termination limits requests after max responses."""
    print("\n--- Example 2: Post-termination Middleware ---\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="TransactionAgent",
            instructions="You help with transaction inquiries.",
            tools=[check_transaction],
            middleware=[PostTerminationMiddleware(max_responses=2)],
        ) as agent,
    ):
        # First request (allowed)
        print("1. First request:")
        result = await agent.run("Check transaction TXN-001")
        print(f"   Agent: {result.text if result.text else 'No response'}\n")

        # Second request (allowed)
        print("2. Second request:")
        result = await agent.run("Check transaction TXN-002")
        print(f"   Agent: {result.text if result.text else 'No response'}\n")

        # Third request (blocked by rate limit)
        print("3. Third request (rate limited):")
        result = await agent.run("Check transaction TXN-003")
        print(f"   Agent: {result.text if result.text else 'No response (rate limited)'}\n")

await run_post_termination_example()

## Key Takeaways

| Technique | When to Use | FSI Example |
|-----------|-------------|-------------|
| `context.terminate = True` + `return` | Block before processing | Prohibited transaction types |
| `await next()` then `terminate` | Allow then limit | Rate limiting |
| Set `context.result` | Custom response | Compliance messages |

## Next Steps
- **Override Result** (notebook 8) - Modify agent responses
- **Shared State** (notebook 9) - Cross-middleware communication